# Modelling cross sections

### Writing C++ extensions

```cpp
//cpp_intro.cc file

#include <torch/extension.h>

torch::Tensor get_rotations(const torch::Tensor &thetas) {
    const auto f = thetas.flatten();
    const auto n = f.numel();
    const auto c = torch::cos(f);
    const auto s = torch::sin(f);
    return torch::stack({c, -s, s, c}).t().view({n, 2, 2});
}

PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
    m.def("get_rotations", &get_rotations, py::call_guard<py::gil_scoped_release>(),
          "Generate 2D rotations given angles thetas");
}
```

In [2]:
import torch
from torch.utils.cpp_extension import load

If you are on Google Colab execute:
```
!pip install Ninja
!add-apt-repository ppa:ubuntu-toolchain-r/test -y
!apt update
!apt upgrade -y
!apt install gcc-9 g++-9
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 100 --slave /usr/bin/g++ g++ /usr/bin/g++-9
```

If on Datalore:
```
!pip install Ninja
!sudo apt-get update && sudo apt-get install cuda-nvcc-11-2

```

Check installation:

In [ ]:
!gcc --version
!g++ --version
!nvcc --version

In [3]:
!mkdir -p build

In [4]:
cpp_intro = load(name='cpp_intro',
             build_directory='./build',
             sources=['cpp_intro.cc'],
             extra_cflags=['-Wall -Wextra -Wpedantic -O3 -std=c++17'],
             verbose=False)

In [5]:
N = 3
PI = 2. * torch.acos(torch.tensor(0.))
thetas = 0.05 * PI * (torch.rand(N) - 0.5) # example of angles in radians
rots = cpp_intro.get_rotations(thetas)
rots

tensor([[[ 0.9998,  0.0180],
         [-0.0180,  0.9998]],

        [[ 1.0000,  0.0088],
         [-0.0088,  1.0000]],

        [[ 0.9982,  0.0606],
         [-0.0606,  0.9982]]])

In [6]:
(rots.matmul(rots.transpose(-1,-2)) - torch.eye(2)).abs().sum()

tensor(1.1921e-07)

### Bremsstrahlung

Bremsstrahlung process corresponds to radiation due to deceleration when two charged particles interact. We will consider here the example of muons. At high energies ($E \geq 1$
TeV) this process contributes to about 40% of the average muon energy loss.

* D. Groom et. al. [Muon stopping power and range tables 10 MeV - 100 TeV](https://pdg.lbl.gov/2014/AtomicNuclearProperties/adndt.pdf)

##### Input:
   * $K$ - The projectile initial kinetic energy, in GeV
   * $q$ - The kinetic energy lost to the photon, in GeV
   * $A$ - The mass number of the target atom, in g/mol
   * $Z$ - The charge number of the target atom.
   * $m_\mu$ - muon rest mass $0.10565839$ GeV
   
##### Output
   The DCS (differential cross-section) in $\text{m}^{2}$/kg:
   
\begin{equation}
\frac{\text{d}\sigma}{\text{d}q} = \alpha Z(2\frac{m_e}{m_\mu})^{2}(\frac{4}{3}(\frac{1}{\nu} - 1) + \nu)(\Phi_\text{in}(\delta) + Z\Phi_n(\delta) ) \frac{N_a}{A \cdot 10^{-3}}
\end{equation}

Here:

\begin{equation}
E = K + m_\mu
\end{equation}

and:

* $\nu = \frac{q}{E}$ is the fraction of the muon's energy transferred to the photon
* $N_a$ the Avogadro number $6.02214199 \cdot 10^{23} \text{mol}^{-1}$
* $m_e$ electron rest mass $0.51099891003 \cdot 10^{-3}$ GeV
* $\alpha$ fine structure constant $1/137.03599976$

We have the contribution from (screened) nucleus:

\begin{equation}
\Phi_n(\delta) = \ln \left( \frac{BZ^{\frac{-1}{3}}(m_\mu + \delta(D_n\sqrt{e} - 2))}{(m_e + \delta \sqrt{e}BZ^{\frac{-1}{3}})D_n} \right),
\end{equation}

where $D_n = 1.54A^{0.27}$ , $B = 182.7$ ($B = 202.4$ for hygrogen), the exponential $e = 2.7181...$ 

and the contributions from atomic electrons:
\begin{equation}
\Phi_\text{in}(\delta) = \ln \left( \frac{m_\mu BZ^{-2/3} \sqrt{e}}{(m_e + \delta  BZ^{-2/3} \sqrt{e})(\frac{m_\mu \delta}{m_e^{2}} + \sqrt{e})} \right),
\end{equation}

where $B = 1429$ ($B = 446$ for hygrogen). 

Both are evaluated at: 

\begin{equation}
\delta = \frac{m_\mu^{2}\nu}{2E(1 - \nu)}
\end{equation}

In [7]:
!git clone https://github.com/grinisrit/noa.git

Cloning into 'noa'...
remote: Enumerating objects: 2469, done.
remote: Counting objects: 100% (517/517), done.
remote: Compressing objects: 100% (310/310), done.
remote: Total 2469 (delta 302), reused 366 (delta 198), pack-reused 1952
Receiving objects: 100% (2469/2469), 22.74 MiB | 461.00 KiB/s, done.
Resolving deltas: 100% (1488/1488), done.


In [9]:
noa_location = 'noa'

In [13]:
muons = load(name='muons',
             build_directory='./build',
             sources=['muons.cc'],
             extra_include_paths=[f'{noa_location}/include'],
             extra_cflags=['-Wall -Wextra -Wpedantic -O3 -std=c++17'],
             verbose=False)

In [14]:
muons_cuda = load(name='muons_cuda',
             build_directory='./build',       
             sources=['muons.cu'],
             extra_include_paths=[f'{noa_location}/include'],
             extra_cflags=['-Wall -Wextra -Wpedantic -O3 -std=c++17'],
             extra_cuda_cflags=['-std=c++17 --extended-lambda'],
             verbose=False) if torch.cuda.is_available() else None

In [15]:
kinetic_energies = torch.linspace(1e-3, 1e6, 10000).double()
recoil_energies = 0.0505 * kinetic_energies

In [16]:
def check_input(tensor, device = torch.device('cpu')):
    assert tensor.dtype == torch.float64
    assert tensor.is_contiguous()
    assert tensor.dim() == 1
    assert tensor.device == device

In [17]:
check_input(kinetic_energies)

In [18]:
brems = muons.bremsstrahlung_dcs(kinetic_energies, recoil_energies)
brems[:5]

tensor([3.5293e-04, 3.9395e-06, 4.0777e-06, 4.1341e-06, 4.1650e-06],
       dtype=torch.float64)

In [19]:
kinetic_energies_gpu = kinetic_energies.cuda()
recoil_energies_gpu = recoil_energies.cuda()

In [20]:
check_input(kinetic_energies_gpu, torch.device('cuda:0'))

In [21]:
brems_gpu = muons_cuda.bremsstrahlung_dcs(kinetic_energies_gpu, recoil_energies_gpu);
brems_gpu[:5]

tensor([3.5293e-04, 3.9395e-06, 4.0777e-06, 4.1341e-06, 4.1650e-06],
       device='cuda:0', dtype=torch.float64)

In [22]:
%timeit muons.bremsstrahlung_dcs(kinetic_energies, recoil_energies)

269 µs ± 2.53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
%timeit muons_cuda.bremsstrahlung_dcs(kinetic_energies_gpu, recoil_energies_gpu)

155 µs ± 5.12 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Modelling high energy DCS

An interpolation model in $\tau = q/K$ is used for $K \geq 10$ GeV and $0.05 \leq \tau \leq 0.95$. For Bremsstrahlung we re-write:

\begin{equation}
\frac{K}{E} \frac{\text{d}\sigma}{\text{d}q} = \frac{K \alpha Z}{q}(2\frac{m_e}{m_\mu})^{2}(\frac{4}{3}(1 - \nu) + \nu^2)(\Phi_\text{in}(\delta) + Z\Phi_n(\delta) ) \frac{N_a}{A \cdot 10^{-3}}
\end{equation}

Setting $X=\ln(\tau)$ and $Y=\ln(1-\tau)$, we fit the model:
\begin{equation}
\ln \left( \frac{K}{E} \frac{\text{d}\sigma}{\text{d}q} \right) = \sum_{i=0}^{6} a_i X^i + b_1 Y + b_2 Y^2
\end{equation}

### Exercises

**(1)** Implement the Bremsstrahlung formula (taking performance into consideration)

**(2)** Fit the polynomial model for the kinetic energies below. How could you improve it?

In [21]:
K = torch.linspace(10, 1e6, 5).double()